# Quick Start Guide

Before we start our tutorial, it is important to understand a SoS script can be executed from command line in batch mode, or interactively in Jupyter notebook. This tutorial is written in Jupyter but you can execute the scripts in batch mode if you save them to disk and execute them using command `sos run` or `sos-runner`. There are some minor differences in executing scripts in these modes so the following cell turns on more output (`-v2`), change default signature mode to `default` (`-s default`), and turn off automatic output preview (`%preview --off`). It then creates some empty files as input files of the sample workflows and remove existing runtime signatures by executing some shell commands (`!cmd`). Please refer to [Chapter Notebook Interface](../documentation/Notebook_Interface.html) of the SoS documentation if you would like to learn more about them.

In [1]:
%set -v2 -s default
%preview --off
!touch mutated.fastq control.fastq ctrl.fastq case.fastq
!cp STAR Rscript ~/.sos/bin/
!sos remove --signature -v 0

Set sos options to "-v2 -s default"


## Multi-Language Notebook

Let us assume that you are a bioinformaticist needed to compare the expression levels between two samples. You can use a Jupyter notebook with SoS kernel to perform the analysis using different script, the trick here is to select the right kernel for each cell. For example, you can run the following cell in bash if you choose `bash` as the kernel.

In [2]:
# index reference genome
STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
    --genomeDir STAR_index
# align reads to the reference genome
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn control.fastq --quantMode GeneCounts \
    --outFileNamePrefix aligned/control
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn mutated.fastq --quantMode GeneCounts \
    --outFileNamePrefix aligned/mutated

Generating STAR_index/chrName.txt
Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq


The first command builds an index of the reference genome in preparation for the latter steps, the second command aligns reads from the first sample to the reference genome, and the third command aligns reads from the second sample to the reference genome. Do not panic if you do not know what these commands are doing, this is just an example.

These commands generate, among other files, two files named ``aligned/control.out.tab`` and ``aligned/mutated.out.tab`` with expression counts of all genes. You then wrote a [R](https://www.r-project.org/) script to analyze the results, something like

In [3]:
control.count <- read.table('aligned/control.out.tab')
mutated.count <- read.table('aligned/mutated.out.tab')
# normalize, compare, output etc, ignored.
pdf('myfigure.pdf')
# plot results
dev.off()

pdf 
  2

## String Interpolation

It is nice to run scripts in different langauges and record results in a notebook script, but the above example is not much different from running two separate scripts from command line. Because different steps of data analysis are naturally connected, it would be good to share information among them. This can be achived by passing (Python) variables defined in the SoS kernel to the subkernels.

For example, if we define the directory for `STAR` index and align output,

In [4]:
INDEXDIR = 'STAR_index'
ALIGNDIR = 'aligned'

INFO: Running interactive_0: 
INFO: Workflow interactive (ID=b138a0442b0e7292) is executed successfully.


we can pass the variable to subkernels through string interpolation, which replaces expressions enbraced by `${` and `}` with the values. For example, the shell script could be written as

In [5]:
# index reference genome
STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
    --genomeDir ${INDEXDIR}
# align reads to the reference genome
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn control.fastq --quantMode GeneCounts \
    --outFileNamePrefix ${ALIGNDIR}/control
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn mutated.fastq --quantMode GeneCounts \
    --outFileNamePrefix ${ALIGNDIR}/mutated

STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
    --genomeDir STAR_index
# align reads to the reference genome
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn control.fastq --quantMode GeneCounts \
    --outFileNamePrefix aligned/control
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn mutated.fastq --quantMode GeneCounts \
    --outFileNamePrefix aligned/mutated
## -- End interpolated command --
Generating STAR_index/chrName.txt
Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq


where expressions such as `${align_dir}` are replaced with their values before the script is interpreted by bash. Similarly, the R script could make use of varialbe `align_dir`.

In [6]:
control.count <- read.table('${ALIGNDIR}/control.out.tab')
mutated.count <- read.table('${ALIGNDIR}/mutated.out.tab')
# normalize, compare, output etc, ignored.
pdf('myfigure.pdf')
# plot results
dev.off()

control.count <- read.table('aligned/control.out.tab')
mutated.count <- read.table('aligned/mutated.out.tab')
# normalize, compare, output etc, ignored.
pdf('myfigure.pdf')
# plot results
dev.off()
## -- End interpolated command --


pdf 
  2

SoS accepts and expands [python format specification and conversion operators](https://docs.python.org/3.5/library/string.html#formatspec) and allows you convert and format the result of expressions. Please refer to [SoS Syntax](../documentation/SoS_Syntax.html) for details.

## Data Exchange among subkernels

String interpolation is useful for composing scripts to be executed by subkernels, but it would be awkward to pass a large amount of information in this way, and it disallows passing information among subkernels.

SoS provides a few `magics` to faciliate data exchange among subkernels. For example, the `%get` magic retrieves information from the SoS kernel to a subkernel, so your shell script could be written as:  

In [7]:
%get INDEXDIR ALIGNDIR
# index reference genome
STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
    --genomeDir $INDEXDIR
# align reads to the reference genome
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn control.fastq --quantMode GeneCounts \
    --outFileNamePrefix $ALIGNDIR/control
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn mutated.fastq --quantMode GeneCounts \
    --outFileNamePrefix $ALIGNDIR/mutated

Generating STAR_index/chrName.txt
Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq


after retriving `INDEXDIR` and `ALIGNDIR` from the SoS kernel and create two native shell variables `INDEXDIR` and `ALIGNDIR`. Similarly, your R script could be written as

In [8]:
%get ALIGNDIR
control.count <- read.table(paste0(ALIGNDIR, '/control.out.tab'))
mutated.count <- read.table(paste0(ALIGNDIR, '/mutated.out.tab'))
# normalize, compare, output etc, ignored.
pdf('myfigure.pdf')
# plot results
dev.off()

pdf 
  2

The subkernels are persistent so the passed variables can be used in later cells using the same kernel, for example, variable `ALIGNDIR` can be used in the Bash kernel in new commands as follows:

In [9]:
echo Align directory is $ALIGNDIR

Align directory is aligned


Note that SoS magics and multiple live subkernels are only supported by Jupyter notebook and cannot be used in SoS batch mode.

## Your first SoS script

The project completed successfully and you needed to archive the scripts for later reference. The Jupyter notebook is a perfect format to record both the commands and the results. However, if you would like to only save the steps of your analysis, you can save the notebook in `.sos` format (`File`->`Download As`->`.sos`) or copy/paste the commands to a single SoS script named ``myanalysis.sos``, with content

In [10]:
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

run:
# index reference genome
STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
    --genomeDir STAR_index

# align reads to the reference genome
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn control.fastq --quantMode GeneCounts \
    --outFileNamePrefix aligned/control
STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
    --readFilesIn mutated.fastq --quantMode GeneCounts \
    --outFileNamePrefix aligned/mutated

R:
control.count <- read.table('aligned/control.out.tab')
mutated.count <- read.table('aligned/mutated.out.tab')
# normalize, compare, output etc, ignored.
pdf('myfigure.pdf')
# plot results
dev.off()

INFO: Running interactive_0: 


Generating STAR_index/chrName.txt
Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq
Generating myfigure.pdf


INFO: Workflow interactive (ID=868b4f6042b50900) is executed successfully.


Here **`run`** and **`R`** are SoS actions that executes the following scripts in `bash` and `R` respectively. The scripts are included verbatim and end after reaching another SoS action or directive. For the sake of time, we created fake `STAR` and `R` commands that simply states the file generated.

The script is executed in Jupyter when the notebook (this documentation) is executed. In command line, you would need to run the script using command

```
% sos run myscript
```

or

```
% myscript.sos
```

if you make `myscript.sos` executable (by running `chmod +x myscript.sos`. 

As you can see, this sos script simply executes the embedded shell and R script and generates the four output files.

Although it is convenient to embed scripts verbatim, it is often clearer to indent the script and write your SoS script as:

In [11]:
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

run:
    # index reference genome
    STAR --runMode genomeGenerate --genomeFastaFile human38.fastq\
        --genomeDir STAR_index

    # align reads to the reference genome
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn control.fastq --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn mutated.fastq --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

# compare expression values
R:
    control.count <- read.table('aligned/control.out.tab')
    mutated.count <- read.table('aligned/mutated.out.tab')
    # normalize, compare, output etc, ignored.
    pdf('myfigure.pdf')
    # plot results
    dev.off()

INFO: Running interactive_0: 


Generating STAR_index/chrName.txt
Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq
Generating myfigure.pdf


INFO: Workflow interactive (ID=585d898c6b6548e9) is executed successfully.


The scripts in this case end with the end of indentation so that you can add some comments for the `R` script without being considered as part of the previous script. The script works identically to the one without indentation.

## Separate scripts into steps

Because the scripts perform different tasks, it is logically clearer to separate them into different **steps**. Actually, because the first reference-generating command is not a data processing step, it makes sense to separate it into two scripts. Now we can insert step headers to the script as follows:

In [12]:
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

[1]
# index reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
        --genomeDir STAR_index

[2]
# align reads to the reference genome
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn control.fastq --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn mutated.fastq --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[3]
# compare expression values
R:
    control.count <- read.table('aligned/control.out.tab')
    mutated.count <- read.table('aligned/mutated.out.tab')
    # normalize, compare, output etc, ignored.
    pdf('myfigure.pdf')
    # plot results
    dev.off()

INFO: Running default_1: index reference genome


Generating STAR_index/chrName.txt


INFO: Running default_2: align reads to the reference genome


Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq


INFO: Running default_3: compare expression values


Generating myfigure.pdf


INFO: Workflow default (ID=160b0c267773e5bc) is executed successfully.


Now, when you execute the script with command

```
sos run myscript
```

SoS will display `default_1`, `default_2` and `default_3` to report progress. The comments after section heads are considered step comments and will also be displayed during execution. 

It is worth noting that you can write SoS workflows in Jupyter notebook as long as you specify a header for each step. For example, you can create a Jupyter notebook with the following cells

In [ ]:
[1]
# index reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
        --genomeDir STAR_index

In [ ]:
[2]
# align reads to the reference genome
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn control.fastq --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn mutated.fastq --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

In [ ]:
[3]
# compare expression values
R:
    control.count <- read.table('aligned/control.out.tab')
    mutated.count <- read.table('aligned/mutated.out.tab')
    # normalize, compare, output etc, ignored.
    pdf('myfigure.pdf')
    # plot results
    dev.off()

and execute the workflow with command

```
sos run myscript.ipynb
```

The notebook can contain markdown cells and cells with other kernels, and sos cells without section header, and the `sos run` (and other) commands will ignore all such cells and only extract and execute workflows defined in the notebook.

## Make the script work for other input files

After a while, before you almost forgot about this analysis, you needed to analyze another pair of samples. You could copy ``myanalysis.sos`` to ``myanalysis2.sos``, change filenames and run it, but an easier way is to change your SoS file to accommodate other input files. This can be done by defining a command line argument and passing files name to a **SoS variable**:

In [13]:
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fastq formats. The first one for control sample
# and the second one for mutated sample.
parameter: fastq_files=['control.fastq', 'mutated.fastq']

[1]
# index reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
        --genomeDir STAR_index

[2]
# align reads to the reference genome
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${fastq_files[0]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${fastq_files[1]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[3]
# compare expression values
R:
    control.count <- read.table('aligned/control.out.tab')
    mutated.count <- read.table('aligned/mutated.out.tab')
    # normalize, compare, output etc, ignored.
    pdf('myfigure.pdf')
    # plot results
    dev.off()

INFO: Running default_1: index reference genome


Generating STAR_index/chrName.txt


INFO: Running default_2: align reads to the reference genome


Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq


INFO: Running default_3: compare expression values


Generating myfigure.pdf


INFO: Workflow default (ID=3b5666a4bc92cb22) is executed successfully.


A command line argument `fastq_files` is defined in with `parameter` keyword. With this definition, you can pass two filenames to variable `fastq_files` from command line

```bash
sos run myanalysis --fastq_files ctrl.fastq case.fastq
```

`${fastq_files[0]}` and `${fastq_files[1]}` in command `STAR --genomeDir ...` will be replaced with their values before the commands are executed. Here `fastq_files[0]` and `fastq_files[1]` are Python expressions that will be evaluated during execution.

In Jupyter notebook, we can execute the previous cell with additional option using magic `%rerun`.

In [14]:
%rerun --fastq_files ctrl.fastq case.fastq

INFO: Running default_1: index reference genome


Generating STAR_index/chrName.txt


INFO: Running default_2: align reads to the reference genome


Generating aligned/control.out.tab from ctrl.fastq
Generating aligned/mutated.out.tab from case.fastq


INFO: Running default_3: compare expression values


Generating myfigure.pdf


INFO: Workflow default (ID=f1873112ddf76541) is executed successfully.


Note that the `parameter` statement is defined before any SOS step, actually in a special `[global]` section although the section is usually ignored in `.sos` format. 

## Ignore steps that do not need to be rerun

Although the SoS script now accepts command line arguments, it is still no more than a compilation of scripts and you immediately realized that it is a waste of time to execute the first command each time. To solve this problem, you can convert the SoS script to a real workflow by telling SoS the input and output of each step:

In [15]:
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fastq formats. The first one for control sample
# and the second one for mutated sample.
parameter: fastq_files=['control.fastq', 'mutated.fastq']

[1]
# create a index for reference genome
output: 'STAR_index/chrName.txt'
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
        --genomeDir STAR_index

[2]
# align the reads to the reference genome
input:    fastq_files
depends:  'STAR_index/chrName.txt'
output:   ['aligned/control.out.tab', 'aligned/mutated.out.tab']
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[0]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[1]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[3]
# compare expression values
output: 'myfigure.pdf'
R:
    control.count <- read.table('${input[0]}')
    mutated.count <- read.table('${input[1]}')
    # normalize, compare, output etc, ignored.
    pdf('${output}')
    # plot results
    dev.off()

INFO: Running default_1: create a index for reference genome
INFO: Step default_1 (index=0) is ignored due to saved signature
INFO: Running default_2: align the reads to the reference genome


Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq


INFO: Running default_3: compare expression values


Generating myfigure.pdf


INFO: Workflow default (ID=65e3d5ba2d00aaeb) is executed successfully.


Here we

- Use **output directive** to specify the expected output of all steps.
- Use **input directive** to specify the input of step 2. Step 1 by default has no input and input for step 3 by default is the output of step 2, its previous step.
- Use **depends directive** to let step 2 depend on the output of step 1.
- Use `${input[0]}` and `${input[1]}` in step 2 and 3 because these steps now have properly-defined `input`. This variable is defined by step input as the input file of the step.

With such information, when you run the same command with another set of input file

```bash
sos run myanalysis --input ctrl.fastq case.fastq
```

SoS will ignore step 1 if this step has been run with output `STAR_index/chrName.txt`. The same happens to step 2 and 3 so all steps will be ignored if you run the script repeatedly with the same input and processing scripts. SoS uses **runtime signature** for each step and will re-run the step if and only if the content or filename of input, output files or the processing scripts are changed.

In [16]:
%rerun --fastq-files ctrl.fastq case.fastq

INFO: Running default_1: create a index for reference genome
INFO: Step default_1 (index=0) is ignored due to saved signature
INFO: Running default_2: align the reads to the reference genome


Generating aligned/control.out.tab from ctrl.fastq
Generating aligned/mutated.out.tab from case.fastq


INFO: Running default_3: compare expression values


Generating myfigure.pdf


INFO: Workflow default (ID=1863e0d7b1f26e5c) is executed successfully.


An added benefit of specifying input and output of steps is it allows you to create an archive of your project with all input, output, and intermediate files. This can be achived using command

```bash
% sos pack -o myanalysis.sar
```

if only one workflow has been executed under the current directory. The output is a compressed archive with extension `.sar` and can be examined and unpacked using command `sos unpack`.

## Use make-rule to define resource-providing steps

Instead of using runtime signature to avoid re-running the first step, we can also make step 1 an optional step that will be executed only necessary. That is to say, we can define this step as an `auxiliary step` that will only be called when the file it **provides** does not exist.

In [17]:
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fastq formats. The first one for control sample
# and the second one for mutated sample.
parameter: fastq_files=['control.fastq', 'mutated.fastq']

[build_index: provides='STAR_index/chrName.txt']
# create a index for reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
        --genomeDir STAR_index

[1]
# align the reads to the reference genome
input:    fastq_files
depends:  'STAR_index/chrName.txt'
output:   ['aligned/control.out.tab', 'aligned/mutated.out.tab']
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[0]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[1]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[2]
# compare expression values
output: 'myfigure.pdf'
R:
    control.count <- read.table('${input[0]}')
    mutated.count <- read.table('${input[1]}')
    # normalize, compare, output etc, ignored.
    pdf('${output}')
    # plot results
    dev.off()

INFO: Adding step build_index with output STAR_index/chrName.txt
INFO: Running build_index: create a index for reference genome
INFO: Step build_index (index=0) is ignored due to saved signature
INFO: Running default_1: align the reads to the reference genome


Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq


INFO: Running default_2: compare expression values


Generating myfigure.pdf


INFO: Workflow default (ID=1c3c60b5b3a1bf3a) is executed successfully.


The new script consists of two steps, and an auxiliary step `build_index` that will only be executed when `STAR_index/chrName.txt` is not available. A slight difference between this version and the previous one is that while the previous version will always execute the first step, this version will not execute it if `STAR_index/chrName.txt` has been generated before in any way, perhaps not by SoS.

In [18]:
%rerun

INFO: Adding step build_index with output STAR_index/chrName.txt
INFO: Running build_index: create a index for reference genome
INFO: Step build_index (index=0) is ignored due to saved signature
INFO: Running default_1: align the reads to the reference genome
INFO: Step default_1 (index=0) is ignored due to saved signature
INFO: Running default_2: compare expression values
INFO: Step default_2 (index=0) is ignored due to saved signature
INFO: Workflow default (ID=1c3c60b5b3a1bf3a) is executed successfully.


## Execute long-running jobs externally

The first step will takes a long time to execute. Instead of executing them by SoS, you might want to submit the commands to a job-queue and be executed and monitored externally. This is especially useful when you need to execute multiple SoS workflows on a cluster-based system. Without going through the details on how to set up your job-queue (see another tutorial for details), it is almost trivial to modify your script to define part of a **step process** to a **task**:

In [19]:
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fastq formats. The first one for control sample
# and the second one for mutated sample.
parameter: fastq_files=['control.fastq', 'mutated.fastq']

[build_index: provides='STAR_index/chrName.txt']
# create a index for reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
        --genomeDir STAR_index

[1]
# align the reads to the reference genome
input:    fastq_files
depends:  'STAR_index/chrName.txt'
output:   ['aligned/control.out.tab', 'aligned/mutated.out.tab']
task:
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[0]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/control
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${input[1]} --quantMode GeneCounts \
        --outFileNamePrefix aligned/mutated

[2]
# compare expression values
output: 'myfigure.pdf'
R:
    control.count <- read.table('${input[0]}')
    mutated.count <- read.table('${input[1]}')
    # normalize, compare, output etc, ignored.
    pdf('${output}')
    # plot results
    dev.off()

INFO: Adding step build_index with output STAR_index/chrName.txt
INFO: Running build_index: create a index for reference genome
INFO: Step build_index (index=0) is ignored due to saved signature
INFO: Running default_1: align the reads to the reference genome
INFO: Step default_1 (index=0) is ignored due to saved signature
INFO: Running default_2: compare expression values
INFO: Step default_2 (index=0) is ignored due to saved signature
INFO: Workflow default (ID=a1fe06faae4de480) is executed successfully.


As you can see, the only difference is the insertion of `task:` directive before `run`. Now, when you execute the command, the `STAR` commands will be executed externally and you can monitor the status of the jobs using your web browser. If no job-queue is set up, the command will be executed in a separate process.

## Execute steps in parallel

Although the two steps of this example have to be executed sequentially, the first step runs the `STAR` command twice on two input files, and can be executed in parallel. You can tell this to SoS by modifying the script as follows

In [20]:
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fastq formats. The first one for control sample
# and the second one for mutated sample.
parameter: fastq_files=['control.fastq', 'mutated.fastq']

[build_index: provides='STAR_index/chrName.txt']
# create a index for reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
        --genomeDir STAR_index

[1]
# align the reads to the reference genome
sample_type = ['control', 'mutated']
input:    fastq_files, group_by='single', paired_with='sample_type'
depends:  'STAR_index/chrName.txt'
output:   "aligned/${_sample_type}.out.tab"

task:     concurrent=True
run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${_input!q} --quantMode GeneCounts \
        --outFileNamePrefix aligned/${_sample_type}

[2]
# compare expression values
output: 'myfigure.pdf'
R:
    control.count <- read.table('${input[0]}')
    mutated.count <- read.table('${input[1]}')
    # normalize, compare, output etc, ignored.
    pdf('${output}')
    # plot results
    dev.off()


INFO: Adding step build_index with output STAR_index/chrName.txt
INFO: Running build_index: create a index for reference genome
INFO: Step build_index (index=0) is ignored due to saved signature
INFO: Running default_1: align the reads to the reference genome


Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq


INFO: Running default_2: compare expression values
INFO: Step default_2 (index=0) is ignored due to saved signature
INFO: Workflow default (ID=95df7eeeafcdaa5c) is executed successfully.


Here we

1. Use option `group_by='single'` to pass input one by one to action. The action will be executed twice with `_input` set to the first and second input file respectively.
2. Define a variable `sample_type` and pair it with input files (option `paired_with`). This will generate a variable `_sample_type` for each input file so `_sample_type` will be `control` for the first input file, and `mutated` for the second.
3. Use `${_input}` and `${_sample_type}` to define partial `output`.
4. Use `${_input!q}` instead of `${_input}` in the script. This is a small trick to shell-quote filenames so that filenames with spaces and other special characters can be properly quoted in shell commands.

Now, if you execute the script with option `-j 2` (2 concurrent processes),

```bash
sos run myanalysis.sos --input control1.fastq control2.fastq -j 2
```

the second step submit two jobs to job-queue, or execute them in two separate processes if a job-queue is not setup.

## Generating a report

In addition to functions such as `run` and `R` that execute scripts in different languages, SoS provides a number of other functions (called actions) that can be used to, for example, generate reports using command `pandoc` or `Rmarkdown`. For example, by adding a step with a `pandoc` action, you can compose a script in Markdown format and generate a nice-looking report in HTML format.

In [2]:
%preview report.html
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

# This script aligns raw reads of a control and a mutated sample
# to the reference genome and compare the expression values
# of the samples at genes A, B and C.

# Two input files in .fastq formats. The first one for control sample
# and the second one for mutated sample.
parameter: fastq_files=['control.fastq', 'mutated.fastq']

[build_index: provides='STAR_index/chrName.txt']
# create a index for reference genome
run:
    STAR --runMode genomeGenerate --genomeFastaFile human38.fastq \
        --genomeDir STAR_index

[1]
# align the reads to the reference genome
sample_type = ['control', 'mutated']
input:    fastq_files, group_by='single', paired_with='sample_type'
depends:  'STAR_index/chrName.txt'
output:   "aligned/${_sample_type}.out.tab"

run:
    STAR --genomeDir STAR_index --outSAMtype BAM SortedByCoordinate \
        --readFilesIn ${_input!q} --quantMode GeneCounts \
        --outFileNamePrefix aligned/${_sample_type}

report:   output='align.md', active=-1
    ## Alignment of reads
    * input file: ${input}
    * output file: ${output}

[2]
# compare expression values
output: 'myfigure.pdf'
R:
    control.count <- read.table('${input[0]}')
    mutated.count <- read.table('${input[1]}')
    # normalize, compare, output etc, ignored.
    pdf('${output}')
    # plot results
    dev.off()

report:   output='analysis.md', active=-1
    ## Statistical analysis
    * input file: ${input}
    * Figure: ${output}

[3]
pandoc: input=['align.md', 'analysis.md'], output='report.html'
    # An analysis of two RNA Seq samples
    

Generating STAR_index/chrName.txt
Generating aligned/control.out.tab from control.fastq
Generating aligned/mutated.out.tab from mutated.fastq
Generating myfigure.pdf


## %preview report.html

> report.html (432 B):

An analysis of two RNA Seq samples Alignment of reads 

  * input file: control.fastq mutated.fastq 



  * output file: aligned/control.out.tab aligned/mutated.out.tab 



Statistical analysis 

  * input file: aligned/control.out.tab aligned/mutated.out.tab 



  * Figure: myfigure.pdf

In this example, partial reports are prepared whenever a step completes, and a final step is used combine all partial reports (with a header) and process it with pandoc.

Please refer to [the report generation tutorial](Generating_Reports.html) for details.

We have showed you multiple versions of the same SoS script, each using more features of SoS. This actually demonstrates one of the advantages of the SoS system, namely you can start using SoS in minutes without knowing any of its advanced features, and gradually improve your script if needed.